In [16]:
import os
import csv
import json
import pandas as pd
import pickle
import plotly

In [20]:
DATALOG_FILENAME_BASE = 'datalog2'
DATALOG_FILENAME_FULL = DATALOG_FILENAME_BASE + '.csv'
DATALOG_PICKLE_NAME = DATALOG_FILENAME_BASE + '.pkl'

# file -I datalog2.csv reveals:
#    datalog2.csv: text/csv; charset=iso-8859-1
#    - this must be a COBB AP Spec, so the program handles it
DATALOG_ENCODING = 'iso-8859-1'

RAW_DATA_FRAME = None

In [23]:
try:
    with open(DATALOG_PICKLE_NAME, 'rb') as log_pkl:
        RAW_DATA_FRAME = pickle.load(log_pkl)
except:
    with open(DATALOG_FILENAME_FULL, 'r', encoding=DATALOG_ENCODING) as log_file:
        RAW_DATA_FRAME = pd.read_csv(log_file)
    
    with open(DATALOG_FILENAME_BASE + '.pkl', 'wb+') as log_pkl:
        pickle.dump(RAW_DATA_FRAME, log_pkl)


In [24]:
print(RAW_DATA_FRAME)

       Time (sec)  AF Correction 1 (%)  AF Correction 3 (%)  \
0           0.000               -14.06                  0.0   
1           0.151               -14.06                  0.0   
2           0.324               -14.06                  0.0   
3           0.508               -14.06                  0.0   
4           0.680               -14.06                  0.0   
...           ...                  ...                  ...   
11128    1969.695                -6.25                  0.0   
11129    1969.872                -6.25                  0.0   
11130    1970.051                -6.25                  0.0   
11131    1970.228                -6.25                  0.0   
11132    1970.408                -6.25                  0.0   

       AF Learning 1 (%)  AF Sens 1 Ratio (AFR)  AF Sens 3 Volts (V)  \
0                    0.0                  14.59                 0.38   
1                    0.0                  14.59                 0.38   
2                    0.0   